In [ ]:
!pip install google-cloud-language>=2.9.1
!pip install --upgrade openai
!pip install xlrd
!pip install -U sentence-transformers
!pip install tqdm

In [ ]:
from google.colab import auth

PROJECT_ID = "helical-rhythm-409706"  # @param {type:"string"}

auth.authenticate_user(project_id=PROJECT_ID)

In [ ]:
!gcloud services enable language.googleapis.com

In [ ]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
from vertexai.generative_models._generative_models import HarmCategory, HarmBlockThreshold, ResponseBlockedError

vertexai.init(project="helical-rhythm-409706", location="us-central1")

def generate_content(prompt):
    config = {
        "max_output_tokens": 2048,
        "temperature": 0.9,
        "top_p": 1
    }

    safety_settings = {
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    }

    model = GenerativeModel("gemini-pro")
    chat = model.start_chat()
    return chat.send_message(prompt, generation_config=config, safety_settings=safety_settings)

In [ ]:
import numpy as np
dataset = np.load('/content/SP-train.npy', allow_pickle=True)
answers = []
questions = []
choices = []
for i in range(len(dataset)):
  answers.append(dataset[i]["answer"])
  questions.append(dataset[i]["question"])
  choices.append(dataset[i]["choice_list"])

import math
from numpy.linalg import norm
import openpyxl
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
wb = openpyxl.load_workbook('/content/EncodedQuestions.xlsx')
ws = wb.active

def findRiddle(Riddle):
  question_emb = model.encode(Riddle)
  A= question_emb
  norm_A = norm(A)
  highest = 0
  for i in range(1,ws.max_row):
    B = [float(x) for x in ws["C"+str(i)].value.split(",")]
    sim = np.dot(A,B)/(norm_A*norm(B))
    if sim > highest:
        highest = sim
        nearest = i

  return [ws["A"+str(nearest)].value, ws["B"+str(nearest)].value]
def farRiddle(Riddle):
  question_emb = model.encode(Riddle)
  A= question_emb
  norm_A = norm(A)
  lowest = 1
  for i in range(1,ws.max_row):
    B = [float(x) for x in ws["C"+str(i)].value.split(",")]
    sim = np.dot(A,B)/(norm_A*norm(B))
    if sim < lowest:
        lowest = sim
        nearest = i
  return [ws["A"+str(nearest)].value, ws["B"+str(nearest)].value]


In [ ]:
choices[6][2]

In [ ]:
#Check Baseline

from difflib import SequenceMatcher
import json
from tqdm import tqdm
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

correct = 0
count = 0
for x in tqdm(range(0,len(questions))):
    prompt = "Please pick the best choice for the brain teaser. Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice text:"
    prompt = prompt + "Question:" + questions[x]+ " \n Choice:" + "\n (A) " + choices[x][0] + "\n (B) " + choices[x][1] + "\n (C) " + choices[x][2] + "\n (D) " +choices[x][3]

    pred = generate_content(prompt)
    pred = pred.candidates[0].content.parts[0].text

    answer = choices[x][0]
    val = similar(choices[x][0], pred)
    if val < similar(choices[x][1], pred):
      answer = choices[x][1]
      val = similar(choices[x][1],pred)
    if val < similar(choices[x][2], pred):
      answer = choices[x][2]
      val = similar(choices[x][2],pred)
    if val < similar(choices[x][3], pred):
      answer = choices[x][3]
      val = similar(choices[x][3],pred)
    if answer  == answers[x]:
      correct += 1
      print("correct", pred)
    else:
      print("incorrect", pred)
    count += 1

print(correct, count, correct/count)



In [ ]:
#Check Baseline + Task Description

from difflib import SequenceMatcher
import json
from tqdm import tqdm
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

correct = 0
count = 0
for x in tqdm(range(0,len(questions))):
    closeRiddle = farRiddle(questions[x])
    prompt = "Please pick the best choice for the sentence puzzler brain teaser (Sentence-type brain teaser where the puzzle defying commonsense is centered on sentence snippets). Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice text:"
    prompt = prompt + "Question:" + questions[x]+ " \n Choice:" + "\n (A) " + choices[x][0] + "\n (B) " + choices[x][1] + "\n (C) " + choices[x][2] + "\n (D) " +choices[x][3] + "\n Example Question with corresponding example answer:" +"\n" +closeRiddle[0] + "\n The answer would be" + closeRiddle[1]

    pred = generate_content(prompt)
    pred = pred.candidates[0].content.parts[0].text

    answer = choices[x][0]
    val = similar(choices[x][0], pred)
    if val < similar(choices[x][1], pred):
      answer = choices[x][1]
      val = similar(choices[x][1],pred)
    if val < similar(choices[x][2], pred):
      answer = choices[x][2]
      val = similar(choices[x][2],pred)
    if val < similar(choices[x][3], pred):
      answer = choices[x][3]
      val = similar(choices[x][3],pred)
    if answer  == answers[x]:
      correct += 1
      print("correct", pred)
    else:
      print("incorrect", pred)
    count += 1

print(correct, count, correct/count)



In [ ]:
#Check Baseline + Task Description + Explain

from difflib import SequenceMatcher
import json
from tqdm import tqdm
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

correct = 0
count = 0
for x in tqdm(range(0,len(questions))):
    closeRiddle = farRiddle(questions[x])
    prompt = "Please pick the best choice for the sentence puzzler brain teaser (Sentence-type brain teaser where the puzzle defying commonsense is centered on sentence snippets). Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice text:"
    prompt = prompt + "Question:" + questions[x]+ " \n Choice:" + "\n (A) " + choices[x][0] + "\n (B) " + choices[x][1] + "\n (C) " + choices[x][2] + "\n (D) " +choices[x][3] + "\n Example Question with corresponding example answer:" +"\n" +closeRiddle[0] + "\n The answer would be" + closeRiddle[1]
    prompt = 'You are a riddle solving expert that explains solutions step by step and responds in raw JSON format following the format { "response": <answer>, "explanation": <explaination> }.' + prompt
    pred = generate_content(prompt)
    pred = pred.candidates[0].content.parts[0].text
    try:
      pred = json.loads(pred)['response']
    except:
      pred = pred

    answer = choices[x][0]
    val = similar(choices[x][0], pred)
    if val < similar(choices[x][1], pred):
      answer = choices[x][1]
      val = similar(choices[x][1],pred)
    if val < similar(choices[x][2], pred):
      answer = choices[x][2]
      val = similar(choices[x][2],pred)
    if val < similar(choices[x][3], pred):
      answer = choices[x][3]
      val = similar(choices[x][3],pred)
    if answer  == answers[x]:
      correct += 1
      print("correct", pred)
    else:
      print("incorrect", pred)
    count += 1

print(correct, count, correct/count)



In [ ]:
#Oneshot

from difflib import SequenceMatcher
import json
import time
from tqdm import tqdm
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


correct = 0
count = 0
for x in tqdm(range(0,len(questions))):
    closeRiddle = findRiddle(questions[x])
    prompt = "Please pick the best choice for the sentence puzzler brain teaser (Sentence-type brain teaser where the puzzle defying commonsense is centered on sentence snippets). Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice text:"
    prompt = prompt + "Question:" + questions[x]+ " \n Choice:" + "\n (A) " + choices[x][0] + "\n (B) " + choices[x][1] + "\n (C) " + choices[x][2] + "\n (D) " +choices[x][3] + "\n Example Question with corresponding example answer:" +"\n" +closeRiddle[0] + "\n The answer would be" + closeRiddle[1]
    pred = generate_content(prompt)
    pred = pred.candidates[0].content.parts[0].text

    answer = choices[x][0]
    val = similar(choices[x][0], pred)
    if val < similar(choices[x][1], pred):
      answer = choices[x][1]
      val = similar(choices[x][1],pred)
    if val < similar(choices[x][2], pred):
      answer = choices[x][2]
      val = similar(choices[x][2],pred)
    if val < similar(choices[x][3], pred):
      answer = choices[x][3]
      val = similar(choices[x][3],pred)
    if answer  == answers[x]:
      correct += 1
      print("correct", pred)
    else:
      print("incorrect", pred)
    count += 1
    time.sleep(1)
print(correct, count, correct/count)



In [ ]:
#Oneshot Explain

from difflib import SequenceMatcher
import json
import time
from tqdm import tqdm
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


correct = 0
count = 0
for x in tqdm(range(0,len(questions))):
    closeRiddle = findRiddle(questions[x])
    prompt = "Please pick the best choice for the sentence puzzler brain teaser (Sentence-type brain teaser where the puzzle defying commonsense is centered on sentence snippets). Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice text:"
    prompt = prompt + "Question:" + questions[x]+ " \n Choice:" + "\n (A) " + choices[x][0] + "\n (B) " + choices[x][1] + "\n (C) " + choices[x][2] + "\n (D) " +choices[x][3] + "\n Example Question with corresponding example answer:" +"\n" +closeRiddle[0] + "\n The answer would be" + closeRiddle[1]

    prompt = 'You are a riddle solving expert that explains solutions step by step and responds in raw JSON format following the format { "response": <answer>, "explanation": <explaination> }.' + prompt
    pred = generate_content(prompt)
    pred = pred.candidates[0].content.parts[0].text
    try:
      pred = json.loads(pred)['response']
    except:
      pred = pred
    answer = choices[x][0]
    val = similar(choices[x][0], pred)
    if val < similar(choices[x][1], pred):
      answer = choices[x][1]
      val = similar(choices[x][1],pred)
    if val < similar(choices[x][2], pred):
      answer = choices[x][2]
      val = similar(choices[x][2],pred)
    if val < similar(choices[x][3], pred):
      answer = choices[x][3]
      val = similar(choices[x][3],pred)
    if answer  == answers[x]:
      correct += 1
      print("correct", pred)
    else:
      print("incorrect", pred)
    count += 1
    time.sleep(1)
print(correct, count, correct/count)

